## Measurement class basics
Objects of the **`measurement class`** are used to save all the information for one single measurement (in contrast to an object of the **`sample`** class that is used for multiple measurements).<br>
A `measurement` object is currently created for spectroscopy and transport(IV charactersisitcs) measurements, with time domain measurement support coming in the near future. Besides **all used instrument settings** it records<br>
* `UUID`, `run`, and `user`
* measurement type (i.e. `spectroscopy`), used function (i.e. `measure_2D()`), and measurement axis
* used `sample` object (in its then form)
* git commit id of `qkit`

The measurement object is saved in the hdf5 file, viewable with `qviewkit` and saved in a separate `.measurement` file in the same folder as the hdf5 file. A saved measurement object can be loaded similar to a data file.

In [8]:
## start qkit and import the necessary classes; here we assume a already configured qkit environment 
import qkit
qkit.cfg["datadir"] = r'C:\Users\weideslab\Documents\QkitLearn\data'
qkit.cfg['fid_init_viewer'] = True
qkit.cfg['fid_scan_hdf'] = True




qkit.start()

from qkit.measure.measurement_class import Measurement
import qkit.core.lib.file_service.file_info_database as fid
f= fid.fid()

Starting QKIT framework ... -> qkit.core.startup
Loading module ... S10_logging.py
Loading module ... S14_setup_directories.py
Loading module ... S16_available_modules.py
Loading module ... S20_check_for_updates.py
Loading module ... S25_info_service.py
Loading module ... S30_qkit_start.py
Loading module ... S65_load_RI_service.py
Loading module ... S70_load_visa.py
Loading module ... S80_load_file_service.py
Loading module ... S85_init_measurement.py
Loading module ... S98_started.py
Loading module ... S99_init_user.py
Initialized the file info database (qkit.fid) in 0.011 seconds.


In `qkit`'s **file information database** there is a dictionary, mapping the **UUID** of the data file to the absolute **path of the saved `.measurement` file**. This can be loaded by creating a **`measurement` object and parsing the abspath**.

In [28]:
f.show() # FIX THE RATING COMPARISON in line 406 in file_info_database.py

,time,datetime,run,name,user,rating,comment
QRT3CL,1618827573,2021-04-19 11:19:33,basic_data_storage,NewFancyData,wridhdhi,10.0,b'New data has been created ....'
QGC7AX,1599563769,2020-09-08 12:16:09,QkitLearn,3D_current_drive_frequency,data,10.0,


In [30]:
meas_file=qkit.fid.measure_db['QFZ34X']#get the corresponding measurement file

In [31]:
type(qkit.fid.measure_db['QFZ34X'])

str

In [25]:
import json
from qkit.measure.json_handler import QkitJSONEncoder, QkitJSONDecoder
from qkit.measure.samples_class import Sample


In [32]:
f1=open(qkit.fid.measure_db['QFZ34X'])
f1
print(json.load(f1, cls = QkitJSONDecoder)['sample'])

{'comment': '', 'name': 'Arbitrary Sample'}


Each folder has 4 files :
1. the h5 file which has all the measurements
2. the log file
3. the measurement file, which has settings, run ID, sample and more details .
4. the set file which has the settings

using he *json* class we see above if the file has some sample data embedded in it . Else it will return Null error.

## Then initialize Measurement object
- Load the file
- extract the details
- set and update them

In [34]:
m = Measurement() #init
# m.load(meas_file)
m.load(meas_file)

In [39]:
#lets se what's in this file
print(m.get_JSON())

{
    "analyzed": false,
    "hdf_relpath": "QFZ34X_VNA_tracedata\\QFZ34X_VNA_tracedata.h5",
    "instruments": {
        "vna": {
            "Average": {
                "content": true,
                "has_setter": true
            },
            "active_trace": {
                "content": null,
                "has_setter": true
            },
            "averages": {
                "content": 1,
                "has_setter": true
            },
            "bandwidth": {
                "content": 200.0,
                "has_setter": true
            },
            "centerfreq": {
                "content": 5000000000.0,
                "has_setter": true
            },
            "channel_index": {
                "content": 1,
                "has_setter": true
            },
            "cw": {
                "content": false,
                "has_setter": true
            },
            "cwfreq": {
                "content": 300000.0,
                "has_setter": true
 

The information in the file is **JSON encoded** (basically a **large dict**) and upon init gets **converted into object attributes**.

In [35]:
user = m.user
run_id = m.run_id
smpl = m.sample

Besides this readout of parameters is also possible to **set an instrument back** to its settings during this measurement to **recreate the mesurement** environment. For this to work the specific **instruments need to be initialized first**. Note we just show 'vna' but this instrument doesnt exist so we gt an error. 

In [43]:
m.update_instrument('vna')

## also possible for all measurements
## m.update_all_instruments()

KeyError: 'instruments'

The entries can be **added or changed** (be careful not to lose information) **and saved again**.

In [20]:
m.analyzed = True
m.rating = 5
m.save(qkit.fid.measure_db['XXXXXX'])

KeyError: 'XXXXXX'

In [25]:
print(m.get_JSON())

{
    "analyzed": true,
    "hdf_relpath": "QGC74O_3D_current_drive_frequency\\QGC74O_3D_current_drive_frequency.h5",
    "instruments": {
        "hp_source": {},
        "mw": {
            "frequency": {
                "content": 7000000000.0,
                "has_setter": true
            },
            "high_power": {
                "content": true,
                "has_setter": true
            },
            "power": {
                "content": 10.0,
                "has_setter": true
            },
            "slope": {
                "content": null,
                "has_setter": true
            },
            "status": {
                "content": true,
                "has_setter": true
            }
        },
        "vna": {
            "Average": {
                "content": true,
                "has_setter": true
            },
            "active_trace": {
                "content": null,
                "has_setter": true
            },
            "averages": 

In [45]:
m.instruments

{'vna': {'Average': {'content': True, 'has_setter': True},
  'active_trace': {'content': None, 'has_setter': True},
  'averages': {'content': 1, 'has_setter': True},
  'bandwidth': {'content': 200.0, 'has_setter': True},
  'centerfreq': {'content': 5000000000.0, 'has_setter': True},
  'channel_index': {'content': 1, 'has_setter': True},
  'cw': {'content': False, 'has_setter': True},
  'cwfreq': {'content': 300000.0, 'has_setter': True},
  'edel_status': {'content': False, 'has_setter': True},
  'nop': {'content': 4001, 'has_setter': True},
  'port1_edel': {'content': 0.0, 'has_setter': True},
  'port2_edel': {'content': 0.0, 'has_setter': True},
  'power': {'content': -40.0, 'has_setter': True},
  'span': {'content': 4000000000.0, 'has_setter': True},
  'startfreq': {'content': 3000000000.0, 'has_setter': True},
  'startpower': {'content': -20.0, 'has_setter': True},
  'stopfreq': {'content': 7000000000.0, 'has_setter': True},
  'stoppower': {'content': -20.0, 'has_setter': True},
  '

## Summary 
The measurement class is used to read, set measurement settings before running it. 
so the following are the functions of measurement class, seen by `help(m_obj)`

Basically do the following :
1. Initialize a measurement object
2. Either load a measurement file using fid, or just set its attributes.
3. set the instrument parameters using the instrument class. 
4. Then update all instruments.
5. save and close the measurement, and do `measure1D()` or "D/ 3D measurements. 